# 분석주제 : 지역 특성과 병원 진료가 어떤 관계가 있을까? 

고려할 지역특성:  인구, 고령화지수, 기후 등 

의료 특성 : 병원의 분포(밀집), 병원 종류 분포 

In [1]:
# 준비 

# colab 드라이브 연결하기 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 패키지 불러오기 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 우선 fm._rebuild() 를 해주고
import matplotlib.font_manager as fm  # 폰트 관련 용도
fm._rebuild()

## 1. 병원 진료 데이터 전처리

### 1.1 가입자 일련번호 기준 데이터 생성 

In [3]:
df1 = pd.read_csv("/content/drive/MyDrive/프로젝트/멀티캠퍼스/프로젝트1/T20_2019_1백만.1.csv", encoding = "euc-kr")
# df2 = pd.read_csv("/content/drive/MyDrive/프로젝트/멀티캠퍼스/프로젝트1/T20_2019_1백만.2.csv", encoding = "euc-kr")
# df3 = pd.read_csv("/content/drive/MyDrive/프로젝트/멀티캠퍼스/프로젝트1/T20_2019_1백만.3.csv", encoding = "euc-kr")

In [4]:
df = pd.concat([df1,df2,df3])
df.head()

,기준년도,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,데이터 기준일자
0,2019,1,104243474,2,12,46,20191104,3,5,M751,M750,1,1,0.25,19240,9600,9640,0,20201222
1,2019,1,107479370,2,12,46,20190806,3,1,E1164,N308,1,1,0.25,145680,72800,72880,0,20201222
2,2019,1,106881883,2,12,46,20190920,3,1,N308,E1164,1,1,0.25,13240,6600,6640,3,20201222
3,2019,1,109043837,2,12,46,20190807,2,1,E1164,N308,7,21,0.25,861590,205810,655780,0,20201222
4,2019,1,103174392,2,12,46,20190318,3,13,H6531,J303,1,1,0.15,48440,14500,33940,3,20201222


In [8]:
# 사람 한 명의 데이터로 합치기 
df_info = df[['가입자 일련번호','성별코드', '연령대코드', '시도코드']]
df_info.drop_duplicates(['가입자 일련번호','시도코드'],keep='first', inplace=True, ignore_index=True)
df_info.head()

,가입자 일련번호,성별코드,연령대코드,시도코드
0,1,2,12,46
1,2,2,10,26
2,3,2,7,41
3,4,1,8,11
4,5,2,7,11


In [5]:
df_group = df.groupby(by=["가입자 일련번호",'시도코드']).sum()
df_group = df_group[['요양일수', '입내원일수','심결요양급여비용총액','총처방일수' ]]
df_group.reset_index(inplace = True)
df_group.head()

,가입자 일련번호,시도코드,요양일수,입내원일수,심결요양급여비용총액,총처방일수
0,1,46,63,80,5076650,211
1,2,26,11,108,319800,7
2,3,41,2,2,217800,7
3,4,11,1,1,32340,3
4,5,11,13,13,372350,46


In [10]:
df_merge = pd.merge(df_info,df_group, how='outer',on=['가입자 일련번호','시도코드'])
df_merge.head()

,가입자 일련번호,성별코드,연령대코드,시도코드,요양일수,입내원일수,심결요양급여비용총액,총처방일수
0,1,2,12,46,63,80,5076650,211
1,2,2,10,26,11,108,319800,7
2,3,2,7,41,2,2,217800,7
3,4,1,8,11,1,1,32340,3
4,5,2,7,11,13,13,372350,46


### 1.2 시도코드 변환

In [19]:
li = list(df['시도코드'].unique())
li.sort()
li

[11, 26, 27, 28, 29, 30, 31, 36, 41, 42, 43, 44, 45, 46, 47, 48, 49]

In [5]:
%%writefile sido.txt

시도코드 | 시도명
11 | 서울시
26 | 부산시
27 | 대구시
28 | 인천시
29 | 광주시
30 | 대전시
31 | 울산시
36 | 세종시
41 | 경기도
42 | 강원도
43 | 충청북도
44 | 충청남도
45 | 전라북도
46 | 전라남도
47 | 경상북도
48 | 경상남도
49 | 제주도

Overwriting sido.txt


In [4]:
df_sido = pd.read_csv("/content/sido.txt", sep = "|")
df_sido

,시도코드,시도명
0,11,서울시
1,26,부산시
2,27,대구시
3,28,인천시
4,29,광주시
5,30,대전시
6,31,울산시
7,36,세종시
8,41,경기도
9,42,강원도


In [5]:
df_sido.columns = ['시도코드', '시도명']

In [6]:
df = pd.merge(df,df_sido, how='outer',on=['시도코드'])
df.head()

,기준년도,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,요양개시일자,서식코드,진료과목코드,주상병코드,부상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,데이터 기준일자,시도명
0,2019,1,104243474,2,12,46,20191104,3,5,M751,M750,1,1,0.25,19240,9600,9640,0,20201222,전라남도
1,2019,1,107479370,2,12,46,20190806,3,1,E1164,N308,1,1,0.25,145680,72800,72880,0,20201222,전라남도
2,2019,1,106881883,2,12,46,20190920,3,1,N308,E1164,1,1,0.25,13240,6600,6640,3,20201222,전라남도
3,2019,1,109043837,2,12,46,20190807,2,1,E1164,N308,7,21,0.25,861590,205810,655780,0,20201222,전라남도
4,2019,1,103174392,2,12,46,20190318,3,13,H6531,J303,1,1,0.15,48440,14500,33940,3,20201222,전라남도


In [7]:
df.drop(['기준년도', '데이터 기준일자','진료내역일련번호','시도코드'], axis = 1, inplace = True)

## 2. EDA 

In [8]:
# 모듈 불러오기 
import plotly.graph_objs as go 
import plotly.express as px

한 명이 여러 번 받은 것을 각각으로 염두해둬도 되는 건지 
한 명이 여러 번 받았어도 한 번으로 쳐야 하는 건지는 잘 모르겠음 
일단은 각각으로 계산했음

In [17]:
df_gender = df.groupby(['시도명', '성별코드']).count()
df_gender = pd.DataFrame(df_gender['가입자 일련번호'])
df_gender.reset_index(inplace= True)
df_gender.head()

,시도명,성별코드,가입자 일련번호
0,강원도,1,53454
1,강원도,2,72081
2,경기도,1,499974
3,경기도,2,624769
4,경상남도,1,133556


In [16]:
# 지역 별 환자 성별 

fig = go.Figure()

fig.add_trace(go.Bar(x = df_gender[df_gender['성별코드']==1]['시도명'], y = df_gender[df_gender['성별코드']==1]['가입자 일련번호'],
                     name = '남자'))
fig.add_trace(go.Bar(x = df_gender[df_gender['성별코드']==2]['시도명'], y = df_gender[df_gender['성별코드']==2]['가입자 일련번호'],
                     name = '여자' ))

fig.update_layout(title = '지역 별 환자 성별', xaxis_title = '지역', yaxis_title = '성별(명)')

fig.show()


In [ ]:
# 지역 별 환자 성별 

fig = go.Figure()

fig.add_trace(go.Bar(x = df_gender[df_gender['성별코드']==1]['시도명'], y = df_gender[df_gender['성별코드']==1]['가입자 일련번호'],
                     name = '남자'))
fig.add_trace(go.Bar(x = df_gender[df_gender['성별코드']==2]['시도명'], y = df_gender[df_gender['성별코드']==2]['가입자 일련번호'],
                     name = '여자' ))

fig.update_layout(title = '지역 별 환자 성별', xaxis_title = '지역', yaxis_title = '성별(명)')

fig.show()

In [18]:
df_type = df.groupby(['시도명', '진료과목코드']).count()
df_type = pd.DataFrame(df_type['가입자 일련번호'])
df_type.reset_index(inplace= True)
df_type.head()

,시도명,진료과목코드,가입자 일련번호
0,강원도,0,2021
1,강원도,1,44607
2,강원도,2,2433
3,강원도,3,3638
4,강원도,4,4634


In [32]:
df_sum = pd.DataFrame(df_type.groupby(['시도명']).sum()['가입자 일련번호'])
df_sum.reset_index(inplace = True)
df_type = pd.merge(df_type,df_sum, how='outer',on=['시도명'])
df_type['비율'] = df_type['가입자 일련번호_x']/df_type['가입자 일련번호_y']

In [65]:
df_type

,시도명,진료과목코드,가입자 일련번호_x,비율,가입자 일련번호_y,가입자 일련번호
0,강원도,0,2021,0.016099,125535,125535
1,강원도,1,44607,0.355335,125535,125535
2,강원도,2,2433,0.019381,125535,125535
3,강원도,3,3638,0.028980,125535,125535
4,강원도,4,4634,0.036914,125535,125535
...,...,...,...,...,...,...
431,충청북도,22,89,0.000588,151408,151408
432,충청북도,23,3774,0.024926,151408,151408
433,충청북도,24,1253,0.008276,151408,151408
434,충청북도,26,3,0.000020,151408,151408


In [20]:
jin = df_type['진료과목코드'].unique

In [66]:
# 지역 별 진료과목 

fig = go.Figure()

for i in jin : 
  fig.add_trace(go.Bar(x = df_type[df_type['진료과목코드']==i]['시도명'], y = df_type[df_type['진료과목코드']==i]['비율'],
                     name = f'진료과목코드{i}'))

fig.update_layout(title = '지역 별 환자 진료과목', xaxis_title = '지역', yaxis_title = '성별(명)', barmode="stack")

fig.show()

In [127]:
df_age = df.groupby(['시도명', '연령대코드']).count()
df_age = pd.DataFrame(df_age['가입자 일련번호'])
df_age.reset_index(inplace= True)
df_age.head()

,시도명,연령대코드,가입자 일련번호
0,강원도,1,7942
1,강원도,2,5235
2,강원도,3,2997
3,강원도,4,2996
4,강원도,5,3477


In [128]:
df_sum = pd.DataFrame(df_age.groupby(['시도명']).sum()['가입자 일련번호'])
df_sum.reset_index(inplace = True)
df_age = pd.merge(df_age,df_sum, how='outer',on=['시도명'])
df_age['비율'] = df_age['가입자 일련번호_x']/df_age['가입자 일련번호_y']
df_age.head()

,시도명,연령대코드,가입자 일련번호_x,가입자 일련번호_y,비율
0,강원도,1,7942,125535,0.063265
1,강원도,2,5235,125535,0.041702
2,강원도,3,2997,125535,0.023874
3,강원도,4,2996,125535,0.023866
4,강원도,5,3477,125535,0.027697


In [77]:
df_age.columns

Index(['시도명', '연령대코드', '가입자 일련번호_x', '가입자 일련번호_y', '비율'], dtype='object')

In [118]:
%%writefile age.txt
연령대코드
0~4세	01
5~9세	02
10~14세	03
15~19세	04
20~24세	05
25~29세	06
30~34세	07
35~39세	08
40~44세	09
45~49세	10
50~54세	11
55~59세	12
60~64세	13
65~69세	14
70~74세	15
75~79세	16
80~84세	17
85~89세	18
90~94세	19
95~99세	20
100세 이상	21
15세미만	22
15~64세	23
65세이상	24
85세이상	25
유아(0~7세)	26
초(8~13세)	27
중(14~16세)	28
고(17~19세)	29
10대이하	30
10대	31
20대	32
30대	33
40대	34
50대	35
60대	36
70대	37
80대	38
90대	39
70대이상	40
80대이상	41

Overwriting age.txt


In [119]:
df_agecode = pd.read_csv('/content/age.txt', sep = '\t')
df_agecode.reset_index(inplace = True)
df_agecode.head()

,index,연령대코드
0,0~4세,1
1,5~9세,2
2,10~14세,3
3,15~19세,4
4,20~24세,5


In [129]:
df_age = pd.merge(df_age, df_agecode, how = "left", on = ['연령대코드'])

In [130]:
age = df_age['index'].unique()

In [136]:
len(age)

18

In [169]:
# # 색 모음 
# import matplotlib.colors as mcolors
# colors = mcolors.CSS4_COLORS
# values = []
# for key, value in colors.items() : 
#   values.append(value)


In [168]:
# 지역 별 환자 연령대 
fig = go.Figure()
count = 0
for i in age : 
  fig.add_trace(go.Bar(x = df_age[df_age['index']==i]['시도명'], y = df_age[df_age['index']==i]['비율'],
                     name = i, marker_color=values[count+45]))
  count +=1

fig.update_layout(title = '지역 별 환자 연령', xaxis_title = '지역', yaxis_title = '연령(명)', barmode="stack" )

fig.show()

세종시가 다른 지역에 비해 인구연령대가 낮은 걸로 알고 있는데 확실히 세종시가 어린 환자들이 많은 것을 알 수 있다. 

In [173]:
# 지역 별 환자 당 심결요양비총액 평균 계산 
df_fee = df.groupby(['시도명', '가입자 일련번호']).sum()
df_fee = df_fee[['요양일수', '입내원일수','심결요양급여비용총액', '심결본인부담금', '심결보험자부담금', '총처방일수']]
df_fee.reset_index(inplace = True)
df_fee.head()

,시도명,가입자 일련번호,요양일수,입내원일수,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수
0,강원도,52,21,28,1565030,380500,1184530,210
1,강원도,79,5,5,140040,41800,98240,9
2,강원도,86,1,1,15690,4700,10990,5
3,강원도,96,18,18,265690,83500,182190,109
4,강원도,185,8,8,138180,41200,96980,26


In [176]:
sido_name = df_fee['시도명'].unique()

In [193]:
result = []
for i in sido_name : 
 result.append(df_fee[df_fee['시도명']==i]['심결요양급여비용총액'].mean()/1000)

df_fee_mean = pd.DataFrame({'시도명':sido, 
                            '비용총액평균' : result})
df_fee_mean.head()

,시도명,비용총액평균
0,강원도,1110.437513
1,경기도,936.654293
2,경상남도,1231.128643
3,경상북도,1238.631761
4,광주시,1164.898653


In [208]:
# 지역 별 환자 당 심결요양비총액 평균
fig = go.Figure()
count = 0

fig.add_trace(go.Bar(x = df_fee_mean['시도명'], y = df_fee_mean['비용총액평균']))

fig.update_layout(title = '지역 별 환자 당 심결요양비총액 평균', xaxis_title = '지역', yaxis_title = '(천원)',xaxis={'categoryorder':'total descending'})

fig.show()

In [207]:
# 지역 별 환자 치료일 요양일수 및 입내원일수 평균 

relax = []
cure = []
for i in sido_name : 
 relax.append(df_fee[df_fee['시도명']==i]['요양일수'].mean())
 cure.append(df_fee[df_fee['시도명']==i]['입내원일수'].mean())

df_day = pd.DataFrame({'시도명':sido, 
                            '요양일수평균' : relax, 
                       '입내원일수평균' : cure})
df_day.head()

,시도명,요양일수평균,입내원일수평균
0,강원도,14.740380,27.000590
1,경기도,13.839278,21.370196
2,경상남도,16.950317,25.480195
3,경상북도,17.370356,27.213096
4,광주시,16.529435,25.371774


In [214]:
# 지역 별 환자 치료일 요양일수 및 입내원일수 평균 
fig = go.Figure()
count = 0

fig.add_trace(go.Bar(x = df_day['시도명'], y = df_day['요양일수평균'], name = '요양일수'))
fig.add_trace(go.Bar(x = df_day['시도명'], y = df_day['입내원일수평균'], name = '입내원일수'))

fig.update_layout(title = '지역 별 환자 치료일 요양일수 및 입내원일수 평균 ', xaxis_title = '지역', yaxis_title = '(일)', xaxis={'categoryorder':'max descending'})

fig.show()

In [241]:
df_pop = pd.read_csv('/content/201912_201912_연령별인구현황_연간.csv', encoding= 'euc-kr')
df_pop.head()

,행정구역,2019년_계_총인구수,2019년_계_연령구간인구수,2019년_계_0~4세,2019년_계_5~9세,2019년_계_10~14세,2019년_계_15~19세,2019년_계_20~24세,2019년_계_25~29세,2019년_계_30~34세,2019년_계_35~39세,2019년_계_40~44세,2019년_계_45~49세,2019년_계_50~54세,2019년_계_55~59세,2019년_계_60~64세,2019년_계_65~69세,2019년_계_70~74세,2019년_계_75~79세,2019년_계_80~84세,2019년_계_85~89세,2019년_계_90~94세,2019년_계_95~99세,2019년_계_100세 이상,2019년_남_총인구수,2019년_남_연령구간인구수,2019년_남_0~4세,2019년_남_5~9세,2019년_남_10~14세,2019년_남_15~19세,2019년_남_20~24세,2019년_남_25~29세,2019년_남_30~34세,2019년_남_35~39세,2019년_남_40~44세,2019년_남_45~49세,2019년_남_50~54세,2019년_남_55~59세,2019년_남_60~64세,2019년_남_65~69세,2019년_남_70~74세,2019년_남_75~79세,2019년_남_80~84세,2019년_남_85~89세,2019년_남_90~94세,2019년_남_95~99세,2019년_남_100세 이상,2019년_여_총인구수,2019년_여_연령구간인구수,2019년_여_0~4세,2019년_여_5~9세,2019년_여_10~14세,2019년_여_15~19세,2019년_여_20~24세,2019년_여_25~29세,2019년_여_30~34세,2019년_여_35~39세,2019년_여_40~44세,2019년_여_45~49세,2019년_여_50~54세,2019년_여_55~59세,2019년_여_60~64세,2019년_여_65~69세,2019년_여_70~74세,2019년_여_75~79세,2019년_여_80~84세,2019년_여_85~89세,2019년_여_90~94세,2019년_여_95~99세,2019년_여_100세 이상
0,전국 (0000000000),"51,849,861","51,849,861","1,845,122","2,321,792","2,299,958","2,659,052","3,319,645","3,490,711","3,158,230","3,912,794","3,882,192","4,501,038","4,359,591","4,307,786","3,765,035","2,545,616","1,972,502","1,623,542","1,103,019","530,317","185,868","45,891","20,160","25,864,816","25,864,816","946,540","1,191,675","1,185,701","1,383,262","1,736,875","1,846,052","1,633,656","1,997,634","1,970,568","2,290,410","2,199,729","2,166,958","1,858,067","1,227,645","918,569","688,481","409,083","156,334","42,658","10,089","4,830","25,985,045","25,985,045","898,582","1,130,117","1,114,257","1,275,790","1,582,770","1,644,659","1,524,574","1,915,160","1,911,624","2,210,628","2,159,862","2,140,828","1,906,968","1,317,971","1,053,933","935,061","693,936","373,983","143,210","35,802","15,330"
1,서울특별시 (1100000000),"9,729,107","9,729,107","300,799","361,894","370,557","449,014","641,964","814,659","725,858","784,101","725,112","835,871","775,779","764,189","700,646","489,156","388,790","295,297","178,836","80,651","30,848","8,842","6,244","4,744,059","4,744,059","154,742","185,592","190,760","230,579","310,553","403,107","361,646","393,184","358,785","414,743","382,449","371,993","332,818","229,978","178,497","133,393","72,707","26,524","8,008","2,405","1,596","4,985,048","4,985,048","146,057","176,302","179,797","218,435","331,411","411,552","364,212","390,917","366,327","421,128","393,330","392,196","367,828","259,178","210,293","161,904","106,129","54,127","22,840","6,437","4,648"
2,부산광역시 (2600000000),"3,413,841","3,413,841","107,963","135,358","128,260","154,980","212,685","223,464","191,915","243,795","239,398","278,384","280,880","302,219","294,417","212,830","159,789","124,321","74,493","32,833","11,419","2,648","1,790","1,675,417","1,675,417","55,362","69,565","66,106","80,681","110,827","117,902","98,521","123,935","121,268","139,061","136,792","145,329","139,889","101,137","73,677","53,998","28,193","9,690","2,520",575,389,"1,738,424","1,738,424","52,601","65,793","62,154","74,299","101,858","105,562","93,394","119,860","118,130","139,323","144,088","156,890","154,528","111,693","86,112","70,323","46,300","23,143","8,899","2,073","1,401"
3,대구광역시 (2700000000),"2,438,031","2,438,031","82,807","105,505","106,587","130,611","164,143","159,498","135,805","170,181","178,142","215,245","216,711","211,538","181,981","126,816","93,006","77,106","50,027","23,031","7,092","1,521",678,"1,205,286","1,205,286","42,512","54,081","55,325","68,964","87,408","86,786","72,542","85,378","88,141","105,935","106,072","103,604","87,908","59,414","41,924","31,905","18,362","6,942","1,613",312,158,"1,232,745","1,232,745","40,295","51,424","51,262","61,647","76,735","72,712","63,263","84,803","90,001","109,310","110,639","107,934","94,073","67,402","51,082","45,201","31,665","16,089","5,479","1,209",520
4,인천광역시 (2800000000),"2,957,026","2,957,026","107,557","134,843","132,842","150,409","192,590","215,018","187,684","233,505","228,91

In [218]:
df_pop.columns

Index(['행정구역', '2019년_계_총인구수', '2019년_계_연령구간인구수', '2019년_계_0~4세',
       '2019년_계_5~9세', '2019년_계_10~14세', '2019년_계_15~19세', '2019년_계_20~24세',
       '2019년_계_25~29세', '2019년_계_30~34세', '2019년_계_35~39세', '2019년_계_40~44세',
       '2019년_계_45~49세', '2019년_계_50~54세', '2019년_계_55~59세', '2019년_계_60~64세',
       '2019년_계_65~69세', '2019년_계_70~74세', '2019년_계_75~79세', '2019년_계_80~84세',
       '2019년_계_85~89세', '2019년_계_90~94세', '2019년_계_95~99세', '2019년_계_100세 이상',
       '2019년_남_총인구수', '2019년_남_연령구간인구수', '2019년_남_0~4세', '2019년_남_5~9세',
       '2019년_남_10~14세', '2019년_남_15~19세', '2019년_남_20~24세', '2019년_남_25~29세',
       '2019년_남_30~34세', '2019년_남_35~39세', '2019년_남_40~44세', '2019년_남_45~49세',
       '2019년_남_50~54세', '2019년_남_55~59세', '2019년_남_60~64세', '2019년_남_65~69세',
       '2019년_남_70~74세', '2019년_남_75~79세', '2019년_남_80~84세', '2019년_남_85~89세',
       '2019년_남_90~94세', '2019년_남_95~99세', '2019년_남_100세 이상', '2019년_여_총인구수',
       '2019년_여_연령구간인구수', '2019년_여_0~4세', '2019년_여_5~9세', '2019년_여_10~14

In [242]:
df_pop_age = df_pop[['행정구역','2019년_계_총인구수','2019년_계_0~4세','2019년_계_5~9세', '2019년_계_10~14세', '2019년_계_15~19세', '2019년_계_20~24세',
       '2019년_계_25~29세', '2019년_계_30~34세', '2019년_계_35~39세', '2019년_계_40~44세',
       '2019년_계_45~49세', '2019년_계_50~54세', '2019년_계_55~59세', '2019년_계_60~64세',
       '2019년_계_65~69세', '2019년_계_70~74세', '2019년_계_75~79세', '2019년_계_80~84세',
       '2019년_계_85~89세']]
df_pop_age.head()

,행정구역,2019년_계_총인구수,2019년_계_0~4세,2019년_계_5~9세,2019년_계_10~14세,2019년_계_15~19세,2019년_계_20~24세,2019년_계_25~29세,2019년_계_30~34세,2019년_계_35~39세,2019년_계_40~44세,2019년_계_45~49세,2019년_계_50~54세,2019년_계_55~59세,2019년_계_60~64세,2019년_계_65~69세,2019년_계_70~74세,2019년_계_75~79세,2019년_계_80~84세,2019년_계_85~89세
0,전국 (0000000000),"51,849,861","1,845,122","2,321,792","2,299,958","2,659,052","3,319,645","3,490,711","3,158,230","3,912,794","3,882,192","4,501,038","4,359,591","4,307,786","3,765,035","2,545,616","1,972,502","1,623,542","1,103,019","530,317"
1,서울특별시 (1100000000),"9,729,107","300,799","361,894","370,557","449,014","641,964","814,659","725,858","784,101","725,112","835,871","775,779","764,189","700,646","489,156","388,790","295,297","178,836","80,651"
2,부산광역시 (2600000000),"3,413,841","107,963","135,358","128,260","154,980","212,685","223,464","191,915","243,795","239,398","278,384","280,880","302,219","294,417","212,830","159,789","124,321","74,493","32,833"
3,대구광역시 (2700000000),"2,438,031","82,807","105,505","106,587","130,611","164,143","159,498","135,805","170,181","178,142","215,245","216,711","211,538","181,981","126,816","93,006","77,106","50,027","23,031"
4,인천광역시 (2800000000),"2,957,026","107,557","134,843","132,842","150,409","192,590","215,018","187,684","233,505","228,917","261,044","256,818","259,776","211,475","127,795","97,219","73,919","49,187","24,152"


In [243]:
del df_pop

In [246]:
df_pop_age.columns = ['시도명', '총인구수'] + list(age)
df_pop_age

,시도명,총인구수,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세
0,전국 (0000000000),"51,849,861","1,845,122","2,321,792","2,299,958","2,659,052","3,319,645","3,490,711","3,158,230","3,912,794","3,882,192","4,501,038","4,359,591","4,307,786","3,765,035","2,545,616","1,972,502","1,623,542","1,103,019","530,317"
1,서울특별시 (1100000000),"9,729,107","300,799","361,894","370,557","449,014","641,964","814,659","725,858","784,101","725,112","835,871","775,779","764,189","700,646","489,156","388,790","295,297","178,836","80,651"
2,부산광역시 (2600000000),"3,413,841","107,963","135,358","128,260","154,980","212,685","223,464","191,915","243,795","239,398","278,384","280,880","302,219","294,417","212,830","159,789","124,321","74,493","32,833"
3,대구광역시 (2700000000),"2,438,031","82,807","105,505","106,587","130,611","164,143","159,498","135,805","170,181","178,142","215,245","216,711","211,538","181,981","126,816","93,006","77,106","50,027","23,031"
4,인천광역시 (2800000000),"2,957,026","107,557","134,843","132,842","150,409","192,590","215,018","187,684","233,505","228,917","261,044","256,818","259,776","211,475","127,795","97,219","73,919","49,187","24,152"
5,광주광역시 (2900000000),"1,456,468","53,932","72,587","74,518","89,913","108,958","98,490","83,055","109,888","113,414","131,152","122,103","111,177","91,802","61,567","51,083","39,612","25,281","12,193"
6,대전광역시 (3000000000),"1,474,870","53,424","68,405","70,004","83,175","106,272","106,357","91,275","109,205","111,289","131,192","124,790","119,079","101,712","66,595","48,720","38,793","26,164","12,689"
7,울산광역시 (3100000000),"1,148,019","46,517","57,854","54,939","61,794","74,942","74,195","68,943","90,800","88,648","102,123","108,074","103,285","83,340","51,311","33,079","23,867","14,389","6,833"
8,세종특별자치시 (3600000000),"340,575","21,998","25,251","22,321","17,716","16,611","19,729","25,644","35,820","33,875","30,597","22,627","19,853","16,584","10,624","7,437","6,008","4,372","2,392"
9,경기도 (4100000000),"13,239,666","525,180","659,810","643,213","712,249","854,474","916,221","847,913","1,077,914","1,074,539","1,215,659","1,131,608","1,067,220","862,325","537,047","411,432","325,840","220,092","105,248"


In [247]:
df_pop_age = df_pop_age[1:]
df_pop_age.sort_values(by = '시도명', inplace = True)
df_pop_age

,시도명,총인구수,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세
10,강원도 (4200000000),"1,541,502","47,889","61,667","64,576","79,223","95,994","85,108","75,113","95,556","102,290","126,002","130,059","140,529","134,610","88,478","69,873","66,118","46,187","21,143"
9,경기도 (4100000000),"13,239,666","525,180","659,810","643,213","712,249","854,474","916,221","847,913","1,077,914","1,074,539","1,215,659","1,131,608","1,067,220","862,325","537,047","411,432","325,840","220,092","105,248"
16,경상남도 (4800000000),"3,362,553","123,552","161,948","157,244","176,713","202,556","185,006","177,106","245,575","249,665","293,615","294,426","290,994","255,896","174,112","128,970","110,412","78,363","40,032"
15,경상북도 (4700000000),"2,665,836","89,397","110,787","107,949","127,799","153,606","140,944","133,168","173,639","177,470","214,063","226,491","238,777","222,331","164,125","122,471","115,492","86,363","42,814"
5,광주광역시 (2900000000),"1,456,468","53,932","72,587","74,518","89,913","108,958","98,490","83,055","109,888","113,414","131,152","122,103","111,177","91,802","61,567","51,083","39,612","25,281","12,193"
3,대구광역시 (2700000000),"2,438,031","82,807","105,505","106,587","130,611","164,143","159,498","135,805","170,181","178,142","215,245","216,711","211,538","181,981","126,816","93,006","77,106","50,027","23,031"
6,대전광역시 (3000000000),"1,474,870","53,424","68,405","70,004","83,175","106,272","106,357","91,275","109,205","111,289","131,192","124,790","119,079","101,712","66,595","48,720","38,793","26,164","12,689"
2,부산광역시 (2600000000),"3,413,841","107,963","135,358","128,260","154,980","212,685","223,464","191,915","243,795","239,398","278,384","280,880","302,219","294,417","212,830","159,789","124,321","74,493","32,833"
1,서울특별시 (1100000000),"9,729,107","300,799","361,894","370,557","449,014","641,964","814,659","725,858","784,101","725,112","835,871","775,779","764,189","700,646","489,156","388,790","295,297","178,836","80,651"
8,세종특별자치시 (3600000000),"340,575","21,998","25,251","22,321","17,716","16,611","19,729","25,644","35,820","33,875","30,597","22,627","19,853","16,584","10,624","7,437","6,008","4,372","2,392"


In [248]:
df_pop_age['시도명'] = sido_name

In [254]:
df_pop_age.dtypes

시도명       object
총인구수      object
0~4세      object
5~9세      object
10~14세    object
15~19세    object
20~24세    object
25~29세    object
30~34세    object
35~39세    object
40~44세    object
45~49세    object
50~54세    object
55~59세    object
60~64세    object
65~69세    object
70~74세    object
75~79세    object
80~84세    object
85~89세    object
dtype: object

In [259]:
# 숫자형 변환
col = df_pop_age.columns[1:]
for i in col : 
  df_pop_age[i] = df_pop_age[i].str.replace(",","").astype(int)

In [260]:
# 지역 별 인구 연령대 비율
fig = go.Figure()
count = 0
for i in age : 
  fig.add_trace(go.Bar(x = df_pop_age['시도명'], y = df_pop_age[i]/df_pop_age['총인구수'],
                     name = i, marker_color=values[count+45]))
  count +=1

fig.update_layout(title = '지역 별 환자 연령', xaxis_title = '지역', yaxis_title = '연령(명)', barmode="stack" )

fig.show()

In [ ]:
# 인구 연령대 데이터 vs 환자 연령대 데이터 

## 기후 
- 기후 변화에 따른 질병 변화 

In [284]:
df_weather = pd.read_csv("/content/extremum_20210928153415.csv", encoding='euc-kr')
df_weather.head()

,\t\t지점번호,지점명,일시,평균기온(℃),최고기온(℃),\t최고기온시각,최저기온(℃),최저기온시각일교차,Unnamed: 8
0,\t\t108,서울,2019-01-01,-5.0,-0.6,15:25,-8.2,6:31,7.6
1,\t\t108,서울,2019-01-02,-4.9,0.2,14:58,-8.8,8:02,9.0
2,\t\t108,서울,2019-01-03,-3.5,3.2,14:36,-8.4,7:26,11.6
3,\t\t108,서울,2019-01-04,-1.1,4.1,15:04,-6.2,7:57,10.3
4,\t\t108,서울,2019-01-05,-2.8,1.1,15:21,-5.5,23:58,6.6


In [285]:
df_weather = df_weather[['지점명', '일시', '평균기온(℃)']]
df_weather.head()

,지점명,일시,평균기온(℃)
0,서울,2019-01-01,-5.0
1,서울,2019-01-02,-4.9
2,서울,2019-01-03,-3.5
3,서울,2019-01-04,-1.1
4,서울,2019-01-05,-2.8


In [277]:
df_weather.dtypes

지점명         object
일시          object
평균기온(℃)    float64
dtype: object

In [286]:
df_weather['일시'] = df_weather['일시'].str.replace("-","")
df_weather = df_weather[:365]

In [291]:
df_disease = df[['요양개시일자','주상병코드','진료과목코드']]
df_disease['주상병코드'] = df_disease['주상병코드'].str[:1]

In [309]:
df_disease.dtypes

요양개시일자     int64
주상병코드     object
진료과목코드     int64
dtype: object

In [295]:
df_typecode = pd.read_excel("/content/컬럼정보_코드 (2).xls")
df_typecode = df_typecode[['코드','명칭']]

In [316]:
df_typecode.columns = ['진료과목코드', '명칭']

In [318]:
df_typecode.dtypes

진료과목코드     int64
명칭        object
dtype: object

In [320]:
df_disease = pd.merge(df_disease, df_typecode, how = 'left', on = ['진료과목코드'])

In [325]:
# df_disease.drop('진료과목코드', axis = 1, inplace = True)
df_disease = df_disease.sort_values(by = "요양개시일자")
df_disease.head()

,요양개시일자,주상병코드,명칭
739528,20190101,A,소아청소년과
1772925,20190101,R,응급의학과
4272540,20190101,J,이비인후과
495853,20190101,A,내과
2968497,20190101,I,신경외과


In [328]:
# 일자 별 진료과목 count 

df_time_type = pd.DataFrame(df_disease.groupby(['요양개시일자', "명칭"]).count())
df_time_type.reset_index(inplace = True)
df_time_type.head()

,요양개시일자,명칭,주상병코드
0,20190101,가정의학과,409
1,20190101,결핵과,1
2,20190101,내과,1071
3,20190101,마취통증의학과,12
4,20190101,비뇨의학과,36


In [ ]:
fig = go.Figure()

for i in age : 
  fig.add_trace(go.Bar(x = df_pop_age['시도명'], y = df_pop_age[i]/df_pop_age['총인구수'],
                     name = i, marker_color=values[count+45]))
  fig.add_trace(go.Scatter(x = df_time_type['요양개시일자'] ))
  count +=1

fig.update_layout(title = '지역 별 환자 연령', xaxis_title = '지역', yaxis_title = '연령(명)', barmode="stack" )

fig.show()